In [440]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [441]:
import copy, math
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

In [442]:
import gepard as g
print(g.__version__)

0.9.10


In [443]:
print(g.__file__)

/home/quantum/gepard/src/gepard/__init__.py


In [444]:
pt = g.dset[75][0]

In [445]:
g.describe_data(g.dset[75])

npt x obs     collab  FTn    id  ref.        
----------------------------------------------
35 x XGAMMA  H1      N/A    75  arXiv:0910.5831
----------------------------------------------
TOTAL = 35


In [446]:
pt.observable, pt.val

('XGAMMA', 2156.0)

In [447]:
class MyTheory(g.PWNormGPD, g.MellinBarnesCFF, g.MellinBarnesTFF, g.DVMP):
    pass

In [448]:
th0 = MyTheory(p=0)

In [449]:
th0.parameters.update({'a2': 0.123, 'ns': 0.15203977400126656, 'al0s': 1.1575112053849788, 'al0g': 1.2473462505404416, 'alps': 0.5312673554988383, 'ms2': 2.740237296159404, 'secs': 0.04939583198250308, 'this': -0.05411781123920072, 'alpg': 0.9999999988492917, 'mg2': 0.11935246209863909, 'secg': 1.342447132772215, 'thig': -0.9997960402412689})

In [450]:
th0.parameters['a2']

0.123

In [451]:
th0.XGAMMA(pt)

42673.19977067151

In [452]:
th0.tff(xi=pt.xi, t=pt.t, Q2=pt.Q2, meson='rho0')

array([-1289.55846534, -3222.41410688,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ])

In [453]:
th0.ImH_V(pt)

-3222.4141068776626

We reproduce this number step by step, as done internally by Gepard.

In [454]:
wce_ar_dvmp = g.wilson.calc_wce(th0, pt.Q2, 'DVMP', 0)

In [455]:
wce_ar_dvmp2 = g.wilson.calc_wce(th0, pt.Q2, 'DVMP', 2)

In [456]:
wce_ar_dvmp[0, 0, :]

array([1.19661056+0.00022386j, 2.11518273+0.0003092j ,
       5.38462485+0.00055892j])

In [457]:
wce_ar_dvmp2[0, 0, :]

array([1.19661056+0.00022386j, 2.11518273+0.0003092j ,
       5.38462485+0.00055892j])

In [458]:
h_prerot = th0.H(pt.xi, pt.t)

In [459]:
h_prerot.shape    #  MBpoints  x  flavor basis (sea quarks, G, u_val, d_val)

(96, 4)

In [460]:
h_prerot[0,:]

array([2.49083853-0.00362085j, 6.56688237-0.01322384j,
       0.        +0.j        , 0.        +0.j        ])

In [461]:
h = np.einsum('fa,ja->jf', th0.frot_rho0_4, h_prerot)    # GPD in evolution basis

In [462]:
wce_ar_dvmp.shape

(3, 96, 3)

In [468]:
reh_aux0, imh_aux0 = th0._mellin_barnes_integral(pt.xi, wce_ar_dvmp, h)
reh_aux2, imh_aux2 = th0._mellin_barnes_integral(pt.xi, wce_ar_dvmp2, h)
reh_aux2 *=th0.parameters['a2']
imh_aux2 *=th0.parameters['a2']
reh_aux = reh_aux0 + reh_aux2
imh_aux = imh_aux0 + imh_aux2

In [469]:
reh_aux, imh_aux

(-70877.26119078598, -177111.69555775655)

In [470]:
astrong = 2 * np.pi * g.qcd.as2pf(th0.p, th0.nf,  pt.Q2, th0.asp[th0.p], th0.r20)

In [471]:
reh, imh = (g.constants.CF * g.constants.F_rho0 * astrong / g.constants.NC / np.sqrt(pt.Q2)
           ) * np.array([reh_aux, imh_aux])

In [472]:
imh

-3222.4141068776626